In [ ]:
import numpy as np
from PIL import Image, ImageSequence
import tifffile as tiff

def tiff_to_gif(tiff_path, gif_path, resize_factor=0.5, fps=25):
    # Open the TIFF file and read all frames
    with tiff.TiffFile(tiff_path) as tif:
        # Extract the frames from the TIFF
        frames = tif.asarray()
    
    # Normalize frames to 8-bit range (0-255)
    min_val = np.min(frames)
    max_val = np.max(frames)
    norm_frames = ((frames - min_val) / (max_val - min_val) * 255).astype(np.uint8)
    
    # Resize frames to reduce GIF size
    resized_frames = []
    for frame in norm_frames:
        img = Image.fromarray(frame)
        if resize_factor != 1.0:
            new_size = (int(img.width * resize_factor), int(img.height * resize_factor))
            img = img.resize(new_size, Image.Resampling.NEAREST)
        resized_frames.append(img)
    
    # Save frames as GIF
    resized_frames[0].save(gif_path, save_all=True, append_images=resized_frames[1:], duration=1000/fps, loop=0, optimize=True)



def stitch_tiff_to_gif(input_tiff_path, output_tiff_path, gif_path, resize_factor=0.5, fps=25):
    # Load input and output TIFF files
    with tiff.TiffFile(input_tiff_path) as input_tif:
        input_frames = input_tif.asarray()
        
    with tiff.TiffFile(output_tiff_path) as output_tif:
        output_frames = output_tif.asarray()
    
    # Ensure both sequences have the same number of frames
    assert input_frames.shape[0] == output_frames.shape[0], "Input and Output TIFF files must have the same number of frames"
    
    # Normalize frames to 8-bit range (0-255) for both input and output
    def normalize_frames(frames):
        min_val = np.min(frames)
        max_val = np.max(frames)
        return ((frames - min_val) / (max_val - min_val) * 255).astype(np.uint8)
    
    norm_input_frames = normalize_frames(input_frames)
    norm_output_frames = normalize_frames(output_frames)
    
    # Stitch frames side by side and resize
    stitched_frames = []
    for input_frame, output_frame in zip(norm_input_frames, norm_output_frames):
        input_img = Image.fromarray(input_frame)
        output_img = Image.fromarray(output_frame)
        
        # Combine images side by side (horizontally)
        stitched_img = Image.new('L', (input_img.width + output_img.width, input_img.height))
        stitched_img.paste(input_img, (0, 0))
        stitched_img.paste(output_img, (input_img.width, 0))
        
        # Resize the stitched image if needed
        if resize_factor != 1.0:
            new_size = (int(stitched_img.width * resize_factor), int(stitched_img.height * resize_factor))
            stitched_img = stitched_img.resize(new_size, Image.Resampling.NEAREST)
        
        stitched_frames.append(stitched_img)
    
    # Save the stitched frames as a GIF
    stitched_frames[0].save(gif_path, save_all=True, append_images=stitched_frames[1:], duration=1000/fps, loop=0, optimize=True)


In [ ]:
stitch_tiff_to_gif('/group/jug/projects/AI4Life/calcium_imaging/DeepCAD/Yoda1_20ms_20%_001.tif', 
                   '/group/jug/edoardo/projects/CalciumImagingDenoising/output/n2v_DeepCAD/Yoda1_20ms_20%_001.tif', 
                   'N2V_2.gif', resize_factor=0.5, fps=100)

In [6]:
tiff_to_gif('/group/jug/projects/AI4Life/calcium_imaging/DeepCAD/Yoda1_20ms_20%_001.tif', 
                   'N2V_2.gif', resize_factor=0.5, fps=100)